In [2]:
df = spark.read.csv("Motor_Vehicle_Collisions_-_Crashes_20241014.csv", header=True, inferSchema=True)
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: string, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: string, NUMBER OF MOTORIST INJURED: string, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [3]:
df.show(5)
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME| BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VE

Geospatial Analysis

In [5]:
collisions_with_location = df.filter((col("LATITUDE").isNotNull()) & (col("LONGITUDE").isNotNull()))

# Count collisions by unique (latitude, longitude) locations
location_counts = collisions_with_location.groupBy("LATITUDE", "LONGITUDE").count()
top_10_locations = location_counts.orderBy(col("count").desc()).limit(10)
top_10_locations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+-----+
|  LATITUDE|  LONGITUDE|count|
+----------+-----------+-----+
|       0.0|        0.0| 4787|
| 40.861862|  -73.91282|  685|
| 40.608757| -74.038086|  670|
| 40.696033|  -73.98453|  646|
|   40.8047|  -73.91243|  597|
|40.6960346|-73.9845292|  587|
| 40.675735|  -73.89686|  575|
| 40.658577|  -73.89063|  513|
|40.7606005|-73.9643142|  474|
|  40.77077|  -73.91727|  467|
+----------+-----------+-----+

In [39]:
from pyspark.sql.functions import col

# Filter for collisions with valid location data
collisions_with_location = df.filter(
    (col("LATITUDE").isNotNull()) & 
    (col("LONGITUDE").isNotNull()) &
    ~((col("LATITUDE") == 0) & (col("LONGITUDE") == 0))  # Exclude (0, 0)
)

# Count collisions by unique (latitude, longitude) locations
location_counts = collisions_with_location.groupBy("LATITUDE", "LONGITUDE").count()

# Get top 10 locations with the most collisions
top_10_locations = location_counts.orderBy(col("count").desc()).limit(10)

# Show results
top_10_locations.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+-----+
|  LATITUDE|  LONGITUDE|count|
+----------+-----------+-----+
| 40.861862|  -73.91282|  685|
| 40.608757| -74.038086|  670|
| 40.696033|  -73.98453|  646|
|   40.8047|  -73.91243|  597|
|40.6960346|-73.9845292|  587|
| 40.675735|  -73.89686|  575|
| 40.658577|  -73.89063|  513|
|40.7606005|-73.9643142|  474|
|  40.77077|  -73.91727|  467|
| 40.820305|  -73.89083|  467|
+----------+-----------+-----+

In [6]:
borough_counts = df.groupBy("BOROUGH").count().orderBy("count", ascending=False)
borough_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+
|      BOROUGH| count|
+-------------+------+
|         NULL|661104|
|     BROOKLYN|466932|
|       QUEENS|392870|
|    MANHATTAN|326661|
|        BRONX|216787|
|STATEN ISLAND| 61409|
|            0|     1|
+-------------+------+

In [8]:
# Filter out rows with null or incorrect values in the BOROUGH column
borough_counts_filtered = df.filter((col("BOROUGH").isNotNull()) & (col("BOROUGH") != "0")) \
    .groupBy("BOROUGH").count().orderBy("count", ascending=False)

# Display the cleaned counts
borough_counts_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+
|      BOROUGH| count|
+-------------+------+
|     BROOKLYN|466932|
|       QUEENS|392870|
|    MANHATTAN|326661|
|        BRONX|216787|
|STATEN ISLAND| 61409|
+-------------+------+

Brooklyn has the highest number of accidents with 466,932, followed by Queens with 392,870 and Manhattan with 326,661. The Bronx recorded 216,787 accidents, while Staten Island had the lowest count at 61,409.

In [10]:
collisions_with_streets = df.filter((col("ON STREET NAME").isNotNull()) & (col("CROSS STREET NAME").isNotNull()))

# Count collisions by intersections (ON STREET NAME and CROSS STREET NAME)
intersection_counts = collisions_with_streets.groupBy("ON STREET NAME", "CROSS STREET NAME").count().orderBy("count", ascending=False)
intersection_counts.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|      ON STREET NAME|   CROSS STREET NAME|count|
+--------------------+--------------------+-----+
|WEST 42 STREET   ...|8 AVENUE         ...|  396|
|FLATBUSH AVENUE  ...|GRAND ARMY PLAZA ...|  372|
|EAST 59 STREET   ...|2 AVENUE         ...|  339|
|BRUCKNER BOULEVAR...|     EAST 138 STREET|  337|
|ROCKAWAY BOULEVAR...|BROOKVILLE BOULEV...|  334|
|WEST FORDHAM ROAD...|MAJOR DEEGAN EXPR...|  323|
|ROCKAWAY BOULEVAR...|BROOKVILLE BOULEVARD|  322|
|TILLARY STREET   ...|FLATBUSH AVENUE E...|  320|
|EAST 57 STREET   ...|3 AVENUE         ...|  306|
|WEST 34 STREET   ...|7 AVENUE         ...|  299|
+--------------------+--------------------+-----+
only showing top 10 rows

The top accident-prone intersections are West 42 Street & 8 Avenue (396 accidents), followed by Flatbush Avenue & Grand Army Plaza (372) and East 59 Street & 2 Avenue (339). Other notable intersections include Bruckner Boulevard & East 138 Street (337) and Rockaway Boulevard & Brookville Boulevard (334).

In [12]:
# Alternatively, count collisions by individual streets
street_counts_filtered = df.filter(df["ON STREET NAME"].isNotNull())
street_counts = street_counts_filtered.groupBy("ON STREET NAME").count().orderBy("count", ascending=False)
street_counts.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|      ON STREET NAME|count|
+--------------------+-----+
|BROADWAY         ...|17277|
|ATLANTIC AVENUE  ...|15322|
|BELT PARKWAY     ...|13543|
|3 AVENUE         ...|12476|
|NORTHERN BOULEVAR...|11965|
|LONG ISLAND EXPRE...| 9928|
|BROOKLYN QUEENS E...| 9743|
|FLATBUSH AVENUE  ...| 9741|
|LINDEN BOULEVARD ...| 9587|
|QUEENS BOULEVARD ...| 9368|
+--------------------+-----+
only showing top 10 rows

The top streets with the highest number of collisions are Broadway (17,277), Atlantic Avenue (15,322), and Belt Parkway (13,543). Other streets with significant collision counts include 3 Avenue (12,476), Northern Boulevard (11,965), Long Island Expressway (9,928), Brooklyn Queens Expressway (9,743), Flatbush Avenue (9,741), Linden Boulevard (9,587), and Queens Boulevard (9,368).

Severity of Collisions

In [14]:
from pyspark.sql import functions as F
# Define severity based on the injury and fatality counts
df = df.withColumn(
    "SEVERITY",
    F.when(
        (col("NUMBER OF PERSONS KILLED") > 0), "Fatality"
    ).when(
        (col("NUMBER OF PERSONS INJURED") > 0) & (col("NUMBER OF PERSONS KILLED") == 0), "Serious Injury"
    ).otherwise("Minor Damage")
)

# Count accidents by severity
severity_counts = df.groupBy("SEVERITY").count().orderBy("count", ascending=False)
severity_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|      SEVERITY|  count|
+--------------+-------+
|  Minor Damage|1626967|
|Serious Injury| 495680|
|      Fatality|   3117|
+--------------+-------+

In [15]:
from pyspark.sql import functions as F

# Define severity based on the injury and fatality counts
df = df.withColumn(
    "SEVERITY",
    F.when(
        (F.col("NUMBER OF PERSONS KILLED") > 0), "Fatality"
    ).when(
        ((F.col("NUMBER OF PERSONS INJURED") > 0) |
         (F.col("NUMBER OF PEDESTRIANS INJURED") > 0) |
         (F.col("NUMBER OF CYCLIST INJURED") > 0) |
         (F.col("NUMBER OF MOTORIST INJURED") > 0)) & 
        (F.col("NUMBER OF PERSONS KILLED") == 0), "Serious Injury"
    ).otherwise("Minor Damage")
)

# Count accidents by severity
severity_counts = df.groupBy("SEVERITY").count().orderBy("count", ascending=False)
severity_counts.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|      SEVERITY|  count|
+--------------+-------+
|  Minor Damage|1626767|
|Serious Injury| 495880|
|      Fatality|   3117|
+--------------+-------+

In [16]:
from pyspark.sql import functions as F

# Define severity based on injuries and fatalities
df = df.withColumn(
    "SEVERITY",
    F.when(
        (F.col("NUMBER OF PERSONS KILLED") > 0) |
        (F.col("NUMBER OF PEDESTRIANS KILLED") > 0) |
        (F.col("NUMBER OF CYCLIST KILLED") > 0) |
        (F.col("NUMBER OF MOTORIST KILLED") > 0), "Fatality"
    ).when(
        ((F.col("NUMBER OF PERSONS INJURED") > 0) |
         (F.col("NUMBER OF PEDESTRIANS INJURED") > 0) |
         (F.col("NUMBER OF CYCLIST INJURED") > 0) |
         (F.col("NUMBER OF MOTORIST INJURED") > 0)) &
        (F.col("NUMBER OF PERSONS KILLED") == 0) &
        (F.col("NUMBER OF PEDESTRIANS KILLED") == 0) &
        (F.col("NUMBER OF CYCLIST KILLED") == 0) &
        (F.col("NUMBER OF MOTORIST KILLED") == 0), "Serious Injury"
    ).otherwise("Minor Damage")
)

# Count accidents by severity
severity_counts = df.groupBy("SEVERITY").count().orderBy("count", ascending=False)
severity_counts.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|      SEVERITY|  count|
+--------------+-------+
|  Minor Damage|1626763|
|Serious Injury| 495878|
|      Fatality|   3123|
+--------------+-------+

The severity analysis of collisions shows that the majority of incidents (1,626,763) resulted in minor damage with no injuries or fatalities, indicating low severity. However, 495,878 collisions involved serious injuries to persons, pedestrians, cyclists, or motorists, highlighting a substantial number of high-impact incidents. Fatal collisions, while relatively rare at 3,123 cases, represent the most severe outcomes and underscore the critical need for targeted safety measures to reduce fatalities and serious injuries on the roads.

In [22]:
from pyspark.sql.functions import col

# Combine all vehicle type columns into one DataFrame
vehicle_type_dfs = []

for col_name in [
    "VEHICLE TYPE CODE 1",
    "VEHICLE TYPE CODE 2",
    "VEHICLE TYPE CODE 3",
    "VEHICLE TYPE CODE 4",
    "VEHICLE TYPE CODE 5"
]:
    vehicle_type_dfs.append(
        severe_collisions.select(col(col_name).alias("VEHICLE_TYPE"), col("SEVERITY"))
    )

# Union all vehicle type DataFrames
all_vehicle_types = vehicle_type_dfs[0]

for df in vehicle_type_dfs[1:]:
    all_vehicle_types = all_vehicle_types.union(df)

# Filter null and unknown vehicle types
all_vehicle_types = all_vehicle_types.filter(
    (col("VEHICLE_TYPE").isNotNull()) & (col("VEHICLE_TYPE") != "UNKNOWN")
)

# Group by vehicle type and severity
vehicle_type_counts = all_vehicle_types.groupBy("VEHICLE_TYPE", "SEVERITY").count() \
    .orderBy("count", ascending=False)

# Display results
vehicle_type_counts.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+------+
|        VEHICLE_TYPE|      SEVERITY| count|
+--------------------+--------------+------+
|               Sedan|Serious Injury|272660|
|Station Wagon/Spo...|Serious Injury|210574|
|   PASSENGER VEHICLE|Serious Injury|133974|
|SPORT UTILITY / S...|Serious Injury| 57294|
|                Bike|Serious Injury| 41281|
|                Taxi|Serious Injury| 23003|
|             BICYCLE|Serious Injury| 15154|
|          4 dr sedan|Serious Injury| 14367|
|       Pick-up Truck|Serious Injury| 13774|
|          Motorcycle|Serious Injury| 10124|
+--------------------+--------------+------+
only showing top 10 rows

In [24]:
from pyspark.sql.functions import col

# Combine all vehicle type columns into one DataFrame
vehicle_type_dfs = []

for col_name in [
    "VEHICLE TYPE CODE 1",
    "VEHICLE TYPE CODE 2",
    "VEHICLE TYPE CODE 3",
    "VEHICLE TYPE CODE 4",
    "VEHICLE TYPE CODE 5"
]:
    vehicle_type_dfs.append(
        severe_collisions.select(col(col_name).alias("VEHICLE_TYPE"), col("SEVERITY"))
    )

# Union all vehicle type DataFrames
all_vehicle_types = vehicle_type_dfs[0]
for df in vehicle_type_dfs[1:]:
    all_vehicle_types = all_vehicle_types.union(df)

# Filter null and unknown vehicle types
all_vehicle_types = all_vehicle_types.filter(
    (col("VEHICLE_TYPE").isNotNull()) & (col("VEHICLE_TYPE") != "UNKNOWN")
)

# Group by vehicle type and severity
vehicle_type_counts = all_vehicle_types.groupBy("VEHICLE_TYPE", "SEVERITY").count()

# Top 10 vehicle types for Serious Injury
print("Top 10 vehicle types for Serious Injury:")
serious_injury_top10 = vehicle_type_counts.filter(col("SEVERITY") == "Serious Injury") \
    .orderBy("count", ascending=False) \
    .limit(10)
serious_injury_top10.show()

# Top 10 vehicle types for Fatality
print("Top 10 vehicle types for Fatality:")
fatality_top10 = vehicle_type_counts.filter(col("SEVERITY") == "Fatality") \
    .orderBy("count", ascending=False) \
    .limit(10)
fatality_top10.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 10 vehicle types for Serious Injury:
+--------------------+--------------+------+
|        VEHICLE_TYPE|      SEVERITY| count|
+--------------------+--------------+------+
|               Sedan|Serious Injury|272660|
|Station Wagon/Spo...|Serious Injury|210574|
|   PASSENGER VEHICLE|Serious Injury|133974|
|SPORT UTILITY / S...|Serious Injury| 57294|
|                Bike|Serious Injury| 41281|
|                Taxi|Serious Injury| 23003|
|             BICYCLE|Serious Injury| 15154|
|          4 dr sedan|Serious Injury| 14367|
|       Pick-up Truck|Serious Injury| 13774|
|          Motorcycle|Serious Injury| 10124|
+--------------------+--------------+------+

Top 10 vehicle types for Fatality:
+--------------------+--------+-----+
|        VEHICLE_TYPE|SEVERITY|count|
+--------------------+--------+-----+
|Station Wagon/Spo...|Fatality| 1101|
|               Sedan|Fatality| 1073|
|   PASSENGER VEHICLE|Fatality|  575|
|SPORT UTILITY / S...|Fatality|  347|
|          Motorcycle|Fatal

The analysis of severe collisions reveals that sedans are the most common vehicle type involved in both serious injuries (272,660) and fatalities (1,073). Station wagons and sport utility vehicles (SUVs) are also highly represented in both categories, with 210,574 serious injuries and 1,101 fatalities for station wagons/SUVs. Other notable contributors include passenger vehicles, which account for 133,974 serious injuries and 575 fatalities, and motorcycles, which rank higher in fatalities (306) than in serious injuries (10,124).

Additionally, bikes and bicycles are significant in serious injuries (41,281 and 15,154 respectively) and fatalities (165). Pick-up trucks and box trucks feature prominently in both lists, with fewer counts compared to sedans and SUVs. 

In [31]:
# Example of reloading the original data
df = spark.read.csv("Motor_Vehicle_Collisions_-_Crashes_20241014.csv", header=True, inferSchema=True)

# Verify columns
print(df.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']

In [32]:
from pyspark.sql.functions import col, sum

# Cast string columns to integers for numerical operations
df = df.withColumn("NUMBER OF PERSONS INJURED", col("NUMBER OF PERSONS INJURED").cast("integer")) \
       .withColumn("NUMBER OF CYCLIST KILLED", col("NUMBER OF CYCLIST KILLED").cast("integer")) \
       .withColumn("NUMBER OF MOTORIST INJURED", col("NUMBER OF MOTORIST INJURED").cast("integer"))

# Filter collisions involving bicycles or pedestrians
bicycle_pedestrian_collisions = df.filter(
    (col("NUMBER OF PEDESTRIANS INJURED") > 0) |
    (col("NUMBER OF PEDESTRIANS KILLED") > 0) |
    (col("NUMBER OF CYCLIST INJURED") > 0) |
    (col("NUMBER OF CYCLIST KILLED") > 0)
)

# Aggregate for injuries and fatalities in collisions involving bicycles or pedestrians
summary_bicycle_pedestrian = bicycle_pedestrian_collisions.select(
    sum("NUMBER OF PERSONS INJURED").alias("Total Injuries"),
    sum("NUMBER OF PERSONS KILLED").alias("Total Fatalities")
)

# Filter collisions involving only vehicles
vehicle_only_collisions = df.filter(
    (col("NUMBER OF PEDESTRIANS INJURED") == 0) &
    (col("NUMBER OF PEDESTRIANS KILLED") == 0) &
    (col("NUMBER OF CYCLIST INJURED") == 0) &
    (col("NUMBER OF CYCLIST KILLED") == 0)
)

# Aggregate for injuries and fatalities in vehicle-only collisions
summary_vehicle_only = vehicle_only_collisions.select(
    sum("NUMBER OF PERSONS INJURED").alias("Total Injuries"),
    sum("NUMBER OF PERSONS KILLED").alias("Total Fatalities")
)

# Display summaries
print("Summary for collisions involving bicycles or pedestrians:")
summary_bicycle_pedestrian.show()

print("Summary for collisions involving only vehicles:")
summary_vehicle_only.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Summary for collisions involving bicycles or pedestrians:
+--------------+----------------+
|Total Injuries|Total Fatalities|
+--------------+----------------+
|        184829|            1880|
+--------------+----------------+

Summary for collisions involving only vehicles:
+--------------+----------------+
|Total Injuries|Total Fatalities|
+--------------+----------------+
|        488611|            1369|
+--------------+----------------+

Collisions Involving Bicycles or Pedestrians:

Total Injuries: 184,829
Total Fatalities: 1,880
These collisions result in a significant number of injuries and a higher fatality count compared to vehicle-only collisions, highlighting the vulnerability of cyclists and pedestrians in traffic incidents.

Collisions Involving Only Vehicles:

Total Injuries: 488,611
Total Fatalities: 1,369
While the total injuries are higher due to the greater frequency of vehicle-only collisions, the fatality rate is notably lower compared to collisions involving bicycles or pedestrians.

Injury and Fatality Patterns

In [35]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Filter for rows with at least one fatality and non-null location data
fatalities_by_location = df.filter(
    (col("NUMBER OF PERSONS KILLED") > 0) &
    (col("LATITUDE").isNotNull()) &
    (col("LONGITUDE").isNotNull())
)

# Handle null boroughs by grouping them under "Unknown"
fatalities_by_location = fatalities_by_location.withColumn(
    "BOROUGH",
    F.when(col("BOROUGH").isNotNull(), col("BOROUGH")).otherwise("Unknown")
)

# Exclude rows where BOROUGH is "Unknown"
fatalities_by_location = fatalities_by_location.filter(col("BOROUGH") != "Unknown")

# Group by borough and count fatalities
fatalities_by_borough = fatalities_by_location.groupBy("BOROUGH").agg(
    F.sum("NUMBER OF PERSONS KILLED").alias("Total_Fatalities")
).orderBy("Total_Fatalities", ascending=False)

# Show results
fatalities_by_borough.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------------+
|      BOROUGH|Total_Fatalities|
+-------------+----------------+
|     BROOKLYN|             614|
|       QUEENS|             524|
|    MANHATTAN|             330|
|        BRONX|             274|
|STATEN ISLAND|              96|
+-------------+----------------+

Brooklyn experiences the highest number of fatalities with 614, followed by Queens with 524, highlighting significant road safety concerns in these boroughs. Manhattan, with 330 fatalities, also reflects the risks associated with its dense traffic and pedestrian activity. The Bronx reports 274 fatalities, while Staten Island, with 96 fatalities, has the lowest count, likely reflecting its smaller population and lower traffic density.

In [36]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Create a column to identify speeding-related accidents
df = df.withColumn(
    "Speeding_Related",
    F.when(
        (col("CONTRIBUTING FACTOR VEHICLE 1") == "Unsafe Speed") |
        (col("CONTRIBUTING FACTOR VEHICLE 2") == "Unsafe Speed") |
        (col("CONTRIBUTING FACTOR VEHICLE 3") == "Unsafe Speed") |
        (col("CONTRIBUTING FACTOR VEHICLE 4") == "Unsafe Speed") |
        (col("CONTRIBUTING FACTOR VEHICLE 5") == "Unsafe Speed"),
        1
    ).otherwise(0)
)

# Calculate the total number of fatal and non-fatal accidents, split by speeding
speeding_fatality_analysis = df.groupBy("Speeding_Related").agg(
    F.sum("NUMBER OF PERSONS KILLED").alias("Total_Fatalities"),
    F.count("COLLISION_ID").alias("Total_Accidents")
).orderBy("Total_Fatalities", ascending=False)

# Show results
speeding_fatality_analysis.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------------+---------------+
|Speeding_Related|Total_Fatalities|Total_Accidents|
+----------------+----------------+---------------+
|               0|            2815|        2089678|
|               1|             434|          36084|
+----------------+----------------+---------------+

Accidents involving speeding are associated with a much higher fatality rate (1.2%) compared to non-speeding-related accidents (0.13%).
While the total number of speeding-related accidents is significantly lower than non-speeding accidents, the fatality rate suggests a strong correlation between speeding and the likelihood of fatalities.

In [38]:
from pyspark.sql.functions import col, desc

# Combine all contributing factor columns into a single column for analysis
contributing_factors = df.select(
    col("CONTRIBUTING FACTOR VEHICLE 1").alias("CONTRIBUTING_FACTOR")
).union(
    df.select(col("CONTRIBUTING FACTOR VEHICLE 2").alias("CONTRIBUTING_FACTOR"))
).union(
    df.select(col("CONTRIBUTING FACTOR VEHICLE 3").alias("CONTRIBUTING_FACTOR"))
).union(
    df.select(col("CONTRIBUTING FACTOR VEHICLE 4").alias("CONTRIBUTING_FACTOR"))
).union(
    df.select(col("CONTRIBUTING FACTOR VEHICLE 5").alias("CONTRIBUTING_FACTOR"))
)

# Filter out null, empty, and "Unspecified" values
contributing_factors = contributing_factors.filter(
    (col("CONTRIBUTING_FACTOR").isNotNull()) &
    (col("CONTRIBUTING_FACTOR") != "") &
    (col("CONTRIBUTING_FACTOR") != "Unspecified")
)

# Group by contributing factors and count occurrences
top_factors = contributing_factors.groupBy("CONTRIBUTING_FACTOR") \
    .count() \
    .orderBy(desc("count")) \
    .limit(5)

# Show the top 5 contributing factors
top_factors.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
| CONTRIBUTING_FACTOR| count|
+--------------------+------+
|Driver Inattentio...|526380|
|Failure to Yield ...|144190|
|Following Too Clo...|136240|
|     Other Vehicular|102450|
|    Backing Unsafely| 86318|
+--------------------+------+

The leading contributing factor in accidents is Driver Inattention/Distraction, accounting for a staggering 526,380 incidents, highlighting the critical need for improved driver focus and awareness. Failure to Yield Right-of-Way ranks second with 144,190 accidents, often occurring at intersections due to poor judgment or lack of caution. Following Too Closely contributed to 136,240 accidents, emphasizing the importance of maintaining safe distances between vehicles. Other Vehicular Causes, a miscellaneous category, accounted for 102,450 accidents, while Backing Unsafely, often due to improper reversing maneuvers, caused 86,318 accidents. 